In [0]:
# thư viện kafka
%pip install kafka-python

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install confluent-kafka

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
with open("/tmp/ca.pem", "r") as f:
    content = f.read()
    print(content)


-----BEGIN CERTIFICATE-----
MIIETTCCArWgAwIBAgIUJUNAXaDT8OIGSuriCSaRnil8trQwDQYJKoZIhvcNAQEM
BQAwQDE+MDwGA1UEAww1MzM5MjQ1ZWYtMjQ5Mi00YWU5LWFjOTQtZmY0MDgwZWIx
NjM5IEdFTiAxIFByb2plY3QgQ0EwHhcNMjUwNTI0MTUzOTI5WhcNMzUwNTIyMTUz
OTI5WjBAMT4wPAYDVQQDDDUzMzkyNDVlZi0yNDkyLTRhZTktYWM5NC1mZjQwODBl
YjE2MzkgR0VOIDEgUHJvamVjdCBDQTCCAaIwDQYJKoZIhvcNAQEBBQADggGPADCC
AYoCggGBAIgARdX63gmdqN5ta+eLnBwymIEnriYmauW8eH08I4UKqMNSEsAUHORO
Iwa67jgA6oQVA5RpWBTLiV9Au4uMNFn0fTW67WwgkROlYEMleoa30RlQ+zIexiOT
QMi20ESV9yBWL8bytd8yPXCW3b/Co4LMyl643+mW/5YbL4yn9cadAtqxP4iSZWre
qCIYpli8uvp3e8we+2zaWvtzZqYhytvpK5bNQp4ipPWyvUs4n/1wEFuSCvMA/UlQ
z79cq2M3QuYMS2nMuNJJpw48ep6Fa3n95xLB7rbtfmn536al2is6IYeF33knEqbV
sUQdZQaspGhBBl+bfEIq84k1K2Lrl4v5Xxe5wh7Y0ticBC1w092hKyavCXJN3q1c
jdPby+YGY7TmPL3fuwweVV6Anh2c6dMVSbvusz9xiEmcpyaYgxzG/HFQCAs8gn6g
O3inLwZ8PT3bU1nHbsZxgPCbawQQJmfE+HRq7CdSeyVwzDSwOk3cr7XVxViaFGw7
I1bBhgCMiwIDAQABoz8wPTAdBgNVHQ4EFgQUl59vPeizVPgDMoHompqwqi9aEeMw
DwYDVR0TBAgwBgEB/wIBADALBgNVHQ8EBAMCAQYwDQYJKoZIhvcNAQEMBQADgg

In [0]:
# weather_producer_aiven.py

import requests
import json
import time
from confluent_kafka import Producer

API_KEY = 'c278dab2865d422c8f7110358251105'
CITY = 'Hanoi'
KAFKA_TOPIC = 'weather_topic'

KAFKA_BROKER = 'weather-kafka-dailong0705-b809.e.aivencloud.com:13010'

producer_config = {
    'bootstrap.servers': KAFKA_BROKER,
    'security.protocol': 'SSL',
    'ssl.ca.location': '/tmp/ca.pem',
    'ssl.certificate.location': '/tmp/service.cert',
    'ssl.key.location': '/tmp/service.key',
    'client.id': 'weather-producer'
}

producer = Producer(producer_config)

def delivery_report(err, msg):
    if err is not None:
        print(f"❌ Delivery failed: {err}")
    else:
        print(f"✅ Sent to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

def get_weather():
    url = f"https://api.weatherapi.com/v1/current.json?key={API_KEY}&q={CITY}&aqi=no"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        current = data['current']
        location = data['location']
        return {
            "time": location['localtime'].split(' ')[1],
            "month": int(location['localtime'].split('-')[1]),
            "year": int(location['localtime'].split('-')[0]),
            "temperature": current['temp_c'],
            "feelslike": current['feelslike_c'],
            "wind": current['wind_kph'],
            "direction": current['wind_dir'],
            "gust": current['gust_kph'],
            "cloud": current['cloud'],
            "humidity": current['humidity'],
            "precipitation": current['precip_mm'],
            "pressure": current['pressure_mb'],
            "weather": current['condition']['text'],
            "label": current['condition']['text']
        }
    else:
        print("❌ API Error:", response.status_code)
        return None

while True:
    weather_data = get_weather()
    if weather_data:
        # Gửi dữ liệu với value là chuỗi JSON bytes
        producer.produce(KAFKA_TOPIC, value=json.dumps(weather_data).encode('utf-8'), callback=delivery_report)
        producer.poll(0)  # Xử lý callback delivery_report

        print("Sent data:")
        for k, v in weather_data.items():
            print(f"     {k}: {v}")
    else:
        print("No data to send")

    time.sleep(3)

    # Đảm bảo xử lý tất cả các callback trước khi thoát vòng lặp
    producer.flush()


Sent data:
     time: 07:52
     month: 5
     year: 2025
     temperature: 21.6
     feelslike: 21.7
     wind: 11.5
     direction: N
     gust: 17.7
     cloud: 100
     humidity: 77
     precipitation: 0.0
     pressure: 1015.0
     weather: Overcast
     label: Overcast
✅ Sent to weather_topic [0] at offset 0
Sent data:
     time: 07:52
     month: 5
     year: 2025
     temperature: 21.6
     feelslike: 21.7
     wind: 11.5
     direction: N
     gust: 17.7
     cloud: 100
     humidity: 77
     precipitation: 0.0
     pressure: 1015.0
     weather: Overcast
     label: Overcast
✅ Sent to weather_topic [0] at offset 1
Sent data:
     time: 07:52
     month: 5
     year: 2025
     temperature: 21.6
     feelslike: 21.7
     wind: 11.5
     direction: N
     gust: 17.7
     cloud: 100
     humidity: 77
     precipitation: 0.0
     pressure: 1015.0
     weather: Overcast
     label: Overcast
✅ Sent to weather_topic [0] at offset 2
Sent data:
     time: 07:52
     month: 5
     year: